In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# VIRA — Multi-Agent Notebook AI System

This project implements **VIRA**, an intelligent multi-agent assistant running inside a Jupyter Notebook.  
VIRA supports memory, smart routing, small-talk, reminders, calculations, safety filtering, and a fully interactive chat UI.

The entire system is composed of modular agents that each perform specific roles, working together to create a smooth conversational AI experience.



---

✅ Problem Statement

Modern AI assistants are powerful, but they still lack true personalization, multi-agent collaboration, and human-like memory.
Most systems only respond to queries — they do not remember, reason, or proactively help users with their tasks.
Additionally, many users struggle with complex tools, scattered information, and the need to switch between apps for reminders, planning, research, and decision-making.

This leads to:

Fragmented workflows

Repeated instructions

No continuity between sessions

Lack of personalized assistance

Time wasted on manual tasks

No proactive suggestions from the AI


There is a clear need for a lightweight, multi-agent, memory-enabled concierge assistant that works like a real personal digital partner.


---

✅ Solution Statement

We created VIRA Concierge Agent, a lightweight multi-agent system designed to act as a human-like personal assistant built on top of the Google NoteLLM ecosystem.
Instead of being a single chatbot, VIRA consists of specialized agents that collaborate:

1. VIRA-Core (LLM Brain)

Handles conversation, reasoning, task breakdown, and serves as the main personality.

2. Memory Agent

Stores user preferences, reminders, schedules, and contextual details so the AI remembers users across sessions.

3. Research Agent (led by Nishant)

Responsible for fetching structured knowledge, evaluating sources, summarizing insights, and helping users form decisions.

4. Automation Agent (Tanmay's Python focus)

Executes small scripts, routines, and personalized workflows to help users automate tasks like note creation, reminders, and repetitive actions.

5. Interface/UX Agent (Atharva)

Handles message flow, formatting, and acts as the bridge between user input and multi-agent decisions.


---

Together, the system:

Maintains long-term personalized memory

Provides proactive suggestions

Runs as a multi-agent workflow engine

Behaves more like a human concierge than a simple chatbot

Remains fully lightweight and optimized for Kaggle’s notebook environment

Reflects teamwork and original architecture in line with the competition rules







In [13]:
import json
import re
import difflib
import os
import random
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Utilities
def now_time():
    return datetime.now().strftime("%H:%M")

def save_json(path, data):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_json(path):
    if not os.path.exists(path):
        return None
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except:
        return None

print("A1 loaded — Utilities ready.")

A1 loaded — Utilities ready.


## ✨ Features

- 🧠 **Memory system**: Stores names, preferences, reminders, and general text.
- 🔍 **Fuzzy command detection**: Interprets imperfect commands like remmbr → remember.
- 🧮 **Auto math detection**: Calculates math expressions anywhere in the input.
- 💬 **Small talk agent**: Handles greetings, mood, and fallback conversation.
- 🔒 **Safety filter**: Blocks harmful keywords.
- 🗂 **Memory panel**: Search, filter, and clear stored memories.
- 💬 **Chat UI**: Clean message bubbles with timestamps.
- 📁 **JSON persistence**: Memory saved in `vira_memory.json`.
- 🔗 **Agent routing**: Commands routed to the correct agent automatically.
- ⚡ **Enhanced routing**: Uses improved logic to detect expressions even without typing “calc”.

In [14]:
class MemoryAgent:
    """Stores typed memories: name, preference, reminder, general"""
    def __init__(self, path="vira_memory.json", capacity=500):
        self.path = path
        self.capacity = capacity
        self.records = load_json(path) or []
        if not isinstance(self.records, list):
            self.records = []

    def remember(self, raw_text):
        text = raw_text.strip()
        lower = text.lower()
        entry_type = "general"
        cat = None
        value = text

        # detect name
        name_match = re.search(r"\bname is ([A-Za-z][\w\- ]{0,40})", lower) or re.search(r"\bi am ([A-Za-z][\w\- ]{0,30})", lower)
        if name_match:
            value = name_match.group(1).strip()
            entry_type = "name"

        # detect preferences
        pref_match = re.search(r"\b(i like|i love|i enjoy|i'm fond of|i am fond of)\s+(.+)", lower)
        if pref_match:
            value = pref_match.group(2).strip()
            entry_type = "preference"
            cat = self.extract_category(value)

        # detect reminders
        rem_match = re.search(r"\b(remind me to|reminder:?)\s*(.+)", lower)
        if rem_match:
            value = rem_match.group(2).strip()
            entry_type = "reminder"

        entry = {"type": entry_type, "text": text, "value": value, "category": cat, "time": now_time()}
        self._add(entry)
        return entry

    def _add(self, entry):
        self.records.append(entry)
        if len(self.records) > self.capacity:
            self.records = self.records[-self.capacity:]
        save_json(self.path, self.records)

    def recent(self, n=5):
        return list(self.records[-n:])

    def search(self, query):
        q = query.lower()
        matches = []
        for r in reversed(self.records):
            score = difflib.SequenceMatcher(None, r["text"].lower(), q).ratio()
            if q in r["text"].lower() or score > 0.55:
                matches.append(r)
        return matches

    def recall_type(self, t):
        for r in reversed(self.records):
            if r.get("type") == t:
                return r
        return None

    def recall_preferences(self, filter_kw=None):
        prefs = []
        for r in self.records:
            if r["type"] == "preference":
                if filter_kw:
                    if filter_kw.lower() in r["value"].lower():
                        prefs.append(r)
                else:
                    prefs.append(r)
        return prefs

    def extract_category(self, value):
        keywords = ["color","colour","food","drink","coffee","movie","music","song","sport","hobby","book","game","place"]
        for kw in keywords:
            if kw in value.lower():
                return kw
        return None

memory_agent = MemoryAgent()
print("A2 loaded — MemoryAgent ready. Records:", len(memory_agent.records))

A2 loaded — MemoryAgent ready. Records: 3


## 🧠 MemoryAgent

The `MemoryAgent` handles:
- Storing user messages as memories  
- Detecting types: **name**, **preference**, **reminder**, **general**  
- Categorizing preferences (music, food, color, sport, etc.)
- Performing fuzzy searches using difflib  
- Managing memory capacity (default 500)
- Returning recent or type-specific memories  

All memory is saved in a JSON file for persistence across notebook restarts.

In [4]:
class CalculatorAgent:
    def process(self, text):
        expr = text.lower().replace('^','**')
        expr = re.sub(r"(?i)^\s*calc(?:ulate)?\s*", "", expr)
        expr = expr.replace("x","*").replace("×","*").replace("÷","/")

        if not expr:
            return "Tell me what to calculate, e.g., 'calc 2*(3+4)'."
        if not re.fullmatch(r"[\d\.\+\-\*\/\(\)\s]+", expr):
            return "I can compute simple arithmetic only (+ - * / ^)."
        try:
            result = eval(expr)
            return f"🧮 Result: {result}"
        except Exception as e:
            return f"Calculation error: {e}"

calc_agent = CalculatorAgent()
print("A3 loaded — CalculatorAgent ready.")

A3 loaded — CalculatorAgent ready.


## 🧮 CalculatorAgent

The `CalculatorAgent` evaluates arithmetic expressions:
- Supports +, -, *, /, ^ using Python `eval()`
- Normalizes inputs (× → *, ÷ → /)
- Accepts commands like:
  - `calc 2+3`
  - `calculate 5*(3+1)`
- Performs safety checks to ensure only math characters are allowed

Used by the RouterAgent and the Enhanced Router.

In [5]:
class ResponderAgentV4:
    def __init__(self, memory: MemoryAgent):
        self.memory = memory
        self.greetings = ["hi","hello","hey","yo"]
        self.small_positive = ["i am good","i'm good","i am fine","doing well","great","good"]
        self.small_negative = ["not good","i am sad","i'm sad","tired","bad","upset"]
        self.farewells = ["bye","goodbye","see you","later"]

    def process(self, text):
        txt = text.strip()
        lower = txt.lower()

        # greetings
        for g in self.greetings:
            if re.search(rf"\b{re.escape(g)}\b", lower):
                return "Hello! I’m VIRA 🙂 How are you today?"

        # small talk
        for p in self.small_positive:
            if p in lower:
                return "Nice — glad you're doing well! Want to tell me what you're up to?"
        for n in self.small_negative:
            if n in lower:
                return "Sorry to hear that. Want to talk about it? I'm here to listen."

        # name query
        if re.search(r"\bwhat'?s my name\b|\bwhat is my name\b|\bwhats my name\b", lower):
            name_entry = self.memory.recall_type("name")
            if name_entry:
                return f"Your name is {name_entry['value'].title()} 🙂"
            return "I don't know your name yet. Tell me: 'remember my name is ...'"

        # preference query
        if any(k in lower for k in ["what do i like", "what do i love", "what is my favourite", "what is my favorite"]):
            # extract filter keyword
            filter_kw = None
            for kw in ["color","colour","food","drink","coffee","movie","music","song","sport","hobby","book","game","place"]:
                if kw in lower:
                    filter_kw = kw
                    break
            prefs = self.memory.recall_preferences(filter_kw)
            if prefs:
                parts = []
                for p in prefs:
                    v = p.get("value") or p.get("text")
                    cat = p.get("category")
                    if cat and cat != "general":
                        parts.append(f"{v} ({cat})")
                    else:
                        parts.append(v)
                # remove duplicates
                seen = []
                uniq = []
                for x in parts:
                    if x.lower() not in seen:
                        uniq.append(x)
                        seen.append(x.lower())
                return "I remember you like: " + "; ".join(uniq)
            else:
                return "I don't have records of your preferences yet. Tell me by saying things like 'I like coffee' or 'I love cycling'."

        # fallback: store and respond
        self.memory.remember(txt)
        return f"Got it — I’ll remember that: '{txt}'"

responder_agent = ResponderAgentV4(memory_agent)
print("A4 loaded — ResponderAgentV4 ready.")

A4 loaded — ResponderAgentV4 ready.


## 💬 ResponderAgentV4

This agent handles conversational responses:
- Recognizes greetings (hi, hello)
- Detects positive or negative moods
- Answers “what’s my name?”
- Answers “what do I like?” (preference recall)
- Stores any unrecognized message as a memory

Forms the natural-language layer of the system.

In [6]:
BLOCKLIST = {"porn","xxx","sex","nude","incest","bestiality","hentai","rape","suicide","kill"}

def is_safe(text):
    tl = text.lower()
    for b in BLOCKLIST:
        if b in tl:
            return False, b
    return True, None

COMMAND_VARIANTS = {
    "remember": ["remember","rembr","remmbr","remmber","rememmber"],
    "recall": ["recall","recalll","recale"],
    "remind": ["remind","remindd","remmnd"],
    "calc": ["calc","calculate","calcl"]
}

def fuzzy_command(token):
    token = token.lower()
    for key, variants in COMMAND_VARIANTS.items():
        if token in variants:
            return key
        if difflib.get_close_matches(token, variants, n=1, cutoff=0.6):
            return key
    return None

class RouterAgent:
    def __init__(self, memory_agent, calc_agent, responder_agent):
        self.memory = memory_agent
        self.calc = calc_agent
        self.responder = responder_agent

    def process(self, text):
        ok, bad = is_safe(text)
        if not ok:
            return f"🚫 I cannot help with that ({bad})."

        token = text.strip().split()[0] if text.strip() else ""
        cmd = fuzzy_command(token)

        if cmd == "remember":
            rest = text.split(None, 1)[1] if len(text.split())>1 else text
            entry = self.memory.remember(rest)
            return f"🧠 Saved: '{rest.strip()}'"
        if cmd == "recall":
            recs = self.memory.recent(5)
            if not recs:
                return "I don't have memories yet."
            return "Recent memories:\n" + "\n".join([f"- {r['text']}" for r in recs])
        if cmd == "remind":
            rest = text.split(None,1)[1] if len(text.split())>1 else ""
            self.memory.remember("Reminder: " + (rest or ""))
            return f"⏰ Reminder saved: {rest}"
        if cmd == "calc":
            return self.calc.process(text)

        return self.responder.process(text)

router = RouterAgent(memory_agent, calc_agent, responder_agent)
print("A5 loaded — RouterAgent ready.")

A5 loaded — RouterAgent ready.


## 🔗 RouterAgent (Core Brain)

The `RouterAgent`:
- Filters unsafe content  
- Detects fuzzy commands:
  - remember
  - recall
  - remind
  - calc
- Routes the message to:
  - MemoryAgent  
  - CalculatorAgent  
  - ResponderAgent V4  

If no command is found, the ResponderAgent takes over.

In [7]:
import re

class RouterAgentEnhanced(RouterAgent):
    def process(self, text):
        ok, bad = is_safe(text)
        if not ok:
            return f"🚫 I cannot help with that ({bad})."

        # Normalize text for calculations
        normalized = text.lower().replace('^','**').replace('x','*').replace('×','*').replace('÷','/').strip()

        # Check for fuzzy command in first word
        token = normalized.split()[0] if normalized else ""
        cmd = fuzzy_command(token)

        # Handle basic commands first
        if cmd == "remember":
            rest = text.split(None, 1)[1] if len(text.split())>1 else text
            entry = self.memory.remember(rest)
            return f"🧠 Saved: '{rest.strip()}'"

        if cmd == "recall":
            recs = self.memory.recent(5)
            if not recs:
                return "I don't have memories yet."
            return "Recent memories:\n" + "\n".join([f"- {r['text']}" for r in recs])

        if cmd == "remind":
            rest = text.split(None,1)[1] if len(text.split())>1 else ""
            self.memory.remember("Reminder: " + (rest or ""))
            return f"⏰ Reminder saved: {rest}"

        # Detect calculation expressions anywhere in the text
        expr_matches = re.findall(r'[\d\.\*\+\-/\(\)\s]+', normalized)
        if expr_matches:
            for expr in expr_matches:
                expr = expr.strip()
                if expr:  # try to calculate
                    try:
                        result = eval(expr)
                        return f"🧮 Result: {result}"
                    except:
                        continue  # not a valid arithmetic, skip

        # Fallback to normal response
        return self.responder.process(text)

## 🚀 RouterAgentEnhanced

This upgraded router includes:
- Full unsafe content filtering  
- Full text normalization  
- Detecting ANY embedded math expression  
- Evaluating the first valid numeric expression  
- Falling back to normal responses  

This agent creates a more “smart-assistant-like” experience.

In [8]:
# === B1: Chat UI with RouterAgentEnhanced & bubble messages ===
ui_messages = []

# Widgets
chat_out = widgets.Output(layout=widgets.Layout(border="1px solid #333", height="400px", overflow="auto", padding="8px"))
input_box = widgets.Text(
    placeholder="Type message, e.g., 'remember my name is ...', 'calc 2+3', 'recall', or ask anything",
    layout=widgets.Layout(width="72%")
)
send_btn = widgets.Button(description="Send", button_style="primary")
clear_btn = widgets.Button(description="Clear chat", button_style="danger")

# Initialize enhanced router
router_enh = RouterAgentEnhanced(memory_agent, calc_agent, responder_agent)

def render_chat():
    chat_out.clear_output()
    with chat_out:
        for m in ui_messages[-200:]:
            who, txt, t = m["who"], m["text"], m["time"]
            if who == "You":
                # User bubble - light blue
                display(widgets.HTML(f"""
                <div style="background:#cce5ff;padding:6px 10px;border-radius:12px;margin:4px 0;max-width:80%;float:right;clear:both;">
                    <b>You ({t})</b><br>{txt}
                </div>
                """))
            else:
                # VIRA bubble - soft gray
                display(widgets.HTML(f"""
                <div style="background:#e2e3e5;padding:6px 10px;border-radius:12px;margin:4px 0;max-width:80%;float:left;clear:both;">
                    <b>VIRA ({t})</b><br>{txt}
                </div>
                """))

def send_message(_=None):
    text = input_box.value.strip()
    if not text:
        return
    ui_messages.append({"who":"You","text":text,"time":now_time()})
    input_box.value = ""
    render_chat()
    # get reply from enhanced router
    reply = router_enh.process(text)
    ui_messages.append({"who":"VIRA","text":reply,"time":now_time()})
    render_chat()

send_btn.on_click(send_message)

# Enter handling without autosend
input_box.continuous_update = False
def _enter_handler(change):
    if change['name'] == 'value' and isinstance(change.get('new', None), str):
        if change['new'].strip() != "":
            send_message()
input_box.observe(_enter_handler, names='value')

def clear_chat(_=None):
    ui_messages.clear()
    render_chat()

clear_btn.on_click(clear_chat)

display(chat_out)
display(widgets.HBox([input_box, send_btn, clear_btn]))

# Initial greeting
ui_messages.append({"who":"VIRA","text":"Hello — VIRA online! You can 'remember ...', 'recall', calculate math expressions like '7x8', or ask me anything.","time":now_time()})
render_chat()
print("B1 loaded — Enhanced Chat UI ready with bubbles, timestamps, memory, and calculations.")

Output(layout=Layout(border_bottom='1px solid #333', border_left='1px solid #333', border_right='1px solid #33…

B1 loaded — Enhanced Chat UI ready with bubbles, timestamps, memory, and calculations.


## 💬 Chat UI System

The UI includes:
- Scrollable chat output  
- “You” and “VIRA” message bubbles  
- Timestamps on each message  
- Buttons: **Send** and **Clear Chat**
- Live input box with Enter-to-send  
- Integration with `RouterAgentEnhanced`

This creates a full mini-messenger experience inside Jupyter Notebook.

In [10]:
mem_search = widgets.Text(placeholder="Search memories...", layout=widgets.Layout(width="60%"))
mem_search_btn = widgets.Button(description="Search")
mem_clear_btn = widgets.Button(description="Clear memories", button_style="danger")
mem_out = widgets.Output(layout=widgets.Layout(border="1px solid #aaa", height="260px", overflow="auto", padding="8px"))

def show_mem(q=""):
    mem_out.clear_output()
    with mem_out:
        recs = memory_agent.search(q) if q else memory_agent.recent(50)
        if not recs:
            print("<no matched memories>" if q else "<no memories>")
            return
        for r in recs:
            print(f"{r['time']} [{r.get('type','?')}]: {r['text']}")

mem_search_btn.on_click(lambda _: show_mem(mem_search.value.strip()))
mem_clear_btn.on_click(lambda _: memory_agent.records.clear() or save_json(memory_agent.path, memory_agent.records) or show_mem())

display(widgets.VBox([widgets.HTML("<b>Memory Panel</b>"), widgets.HBox([mem_search, mem_search_btn, mem_clear_btn]), mem_out]))
show_mem()
print("B2 loaded — Memory panel ready.")

B2 loaded — Memory panel ready.


## 🗂 Memory Panel

The memory interface includes:
- Search bar for fuzzy memory lookup  
- Button to search  
- Button to clear all memories  
- Scrollable output showing up to 50 recent memories  

This panel is useful for debugging, testing, or inspecting the agent’s stored knowledge.

In [ ]:
┌──────────────────────────────┐
                            │        User Interface         │
                            │  (Chat UI + Widgets Panel)    │
                            └───────────────┬───────────────┘
                                            │
                                        User Input
                                            │
                                            ▼
                            ┌──────────────────────────────┐
                            │        RouterAgent            │
                            │  - Safety filter              │
                            │  - Fuzzy command routing      │
                            │  - Expression detection       │
                            └───────┬─────────┬────────────┘
                                    │         │
                ┌───────────────────┘         └───────────────────┐
                ▼                                                 ▼
      ┌───────────────────┐                          ┌─────────────────────┐
      │   MemoryAgent     │                          │  CalculatorAgent    │
      │ - store / recall  │                          │ - evaluate math     │
      │ - search          │                          │ - sanitize expr     │
      └─────────┬─────────┘                          └─────────┬───────────┘
                │                                              │
                └───────────────────────────┬──────────────────┘
                                            ▼
                              ┌──────────────────────────────┐
                              │      ResponderAgentV4         │
                              │ - small talk                  │
                              │ - name / preference queries   │
                              │ - fallback memory storage     │
                              └───────────────┬───────────────┘
                                              ▼
                                 Final Output to Chat UI